In [2]:
# load required libraries and set working directory
import sys
import os
import csv
import re
from datetime import datetime

os.chdir("C:/Users/Adam/Documents/JMSF/Python Scripts/PeptideDomainMapper")

primaryIndexHeader = "row_id"
idRegEx = r'.*'

reportFile = "tmd_overlapping_peptides.txt"

pepFile = "peptides.txt"
tmdFile = "TMDs.txt"

outString = ""

def tab2dict(tableName, primaryIndexHeader, idRegEx):
    with open(tableName, encoding = "ISO-8859-1", newline='' ) as f:
        
        reader=csv.reader(f,delimiter= "\t")
        headers = next(reader)
        #print(headers)
        numheaders = len(headers)
        columnindices = list(range(0, numheaders))
        
        header_indices = dict(zip(headers, columnindices))
        
        f.seek(0)
        next(f) # skip headings
        reader=csv.reader(f,delimiter='\t')

        tabdict = {}
        for line in reader:
            
            lenDiff = len(headers) - len(line)
            
            line += [""] * lenDiff
            
            identifiers = re.findall(idRegEx, line[header_indices[primaryIndexHeader]], re.IGNORECASE)         

            for identifier in identifiers:
                tabdict[identifier.lower()] = {}
                tabdict[identifier.lower()]["dict"] = dict(zip(headers, line))
                tabdict[identifier.lower()]["list"] = line
    
    result = {}
    result["headers"] = headers
    result["tabdict"] = tabdict
    result["header_indices"] = header_indices
    result["tableName"] = tableName
    return result

def checkOverlap (domain1coords, domain2coords, protSeq):
    
    s1 = domain1coords["start"]
    e1 = domain1coords["end"]
    s2 = domain2coords["start"]
    e2 = domain2coords["end"]
      
    #print(str(s2) + " " + str(e2))
    #s1 and e1 are the start and end coordinate of domain 1 and s2 and e2 are the coordinates of the second domain
    
    overlap = 0
    overlap_seq = ""
    
    if s1 == s2 and e1 == e2: #the two domains are identical
        overlap = e1-s1
        overlap_start = s1
        overlap_end = e1
        
    if s1 < s2 and e1 > e2: #domain 2 is an internal subset of domain 1
        overlap = e2-s2
        overlap_start = s2
        overlap_end = e2
        
    if s1 > s2 and e1 < e2: #domain 1 is an internal subset of domain 2
        overlap = e1-s1
        overlap_start = s1
        overlap_end = e1
        
    if s1 < s2 and e1 < e2: #domain 1 spans the left edge of domain 2
        overlap = e1-s2
        overlap_start = s2
        overlap_end = e1
        
    if s1 > s2 and e1 > e2: #domain 1 spans the right edge of domain 2
        overlap = e2-s1
        overlap_start = s1
        overlap_end = e2
    
    if overlap > 0:
        result = {}
        result["overlap_distance"] = overlap
        result["overlap_seq"] = protSeq[overlap_start-1:overlap_end-1]
        #print("hit: " + result["overlap_seq"])
        return result
    else:
        return False

pep = tab2dict(pepFile, primaryIndexHeader, idRegEx)
tmd = tab2dict(tmdFile, "protein_id", idRegEx)
pep_coords = {}

outLines = []

outLines.append("\t".join(pep["headers"]).rstrip() + "\tPeptide Seq\tTMD Seq\tOverlap Seq")

for row in pep["tabdict"]:
    protID = pep["tabdict"][row]["dict"]["protein_id"].lower()
    protSeq = pep["tabdict"][row]["dict"]["protein_seq"]
    pep_coordstring = pep["tabdict"][row]["dict"]["pep_coordinates"]
    
    pep_coordparts = pep_coordstring.replace("[", "").replace("]", "").replace(";","").split("-")
    pep_coords["start"] = int(pep_coordparts[0])
    pep_coords["end"] = int(pep_coordparts[1])
    
    pepSeq = protSeq[pep_coords["start"]-1:pep_coords["end"]-1]
    
    
    #print(pep_coords)
    
    if protID in tmd["tabdict"]:
    
        tmds = tmd["tabdict"][protID]["dict"]
        #print(tmds)
        spansTMD = 0
        overlapSeq = ""
        tmd_coords = {}

        for t in range(1,31):
            tmd_key = "tmd_" + str(t)
            if tmds[tmd_key] == "":
                break
            tmd_coord_parts = tmds[tmd_key].split("-")
            tmd_coords["start"] = int(tmd_coord_parts[0])
            tmd_coords["end"] = int(tmd_coord_parts[1])
            tmdSeq = protSeq[tmd_coords["start"]-1:tmd_coords["end"]-1]
            
            overlapResult = checkOverlap(pep_coords, tmd_coords, protSeq)
            if overlapResult is not False:
                if overlapResult["overlap_distance"] > 1:
                    spansTMD = 1
                    overlapSeq = overlapResult["overlap_seq"]
                    print(overlapSeq)
                    
                    outLines.append("\t".join(pep["tabdict"][row]["list"]).rstrip() + "\t" + pepSeq + "\t" + tmdSeq + "\t" + overlapSeq)
                    #outLines.append(overlapSeq)
        #if spansTMD == 1:
            

output = "\n".join(outLines)     
        
fout = open(reportFile, 'w')
fout.write(output)
fout.close()

IG
AALGSATQAAQ
VGLSYAAVPL
VHLATIP
IGAITSGPIAEF
LESLGGL
ILAYGS
IFQ
TFFASYL
SI
GFILL
WFS
VLAVHGF
IAASSIQIILGFSQLWA
LTTLNQF
INNFI
LA
VIAAGLAVGLASIGPGVGQGTA
IAPVYTAE
APIAVP
TILQ
VHGILGSSI
SIVPVV
IIGAFAGALTGAITTPLDV
IW
VGVYAIY
VGL
SLGQASPSLSA
IAYSSVSH
ASI
LSLIIG
TLLQ
VGGFFV
GLLIQLLSA
FLLGVIP
ALTPWL
GLTLT
LVFLP
WSSF
HIHSLIIF
SNV
LFVYG
VPVGVAGIVGIGTV
FVPIVHVIAG
TFGLVYTVY
FIALI
TLLQ
LIFLTLFVVALS
AVGNILGYAAGS
LLGV
LLAGILL
LLSPIGGL
ASLALPG
IAYSSVSH
PFS
IQ
TFGLVYTVY
ILHVF
LIL
VLL
ISVGFNAAAS
LF
PVVGIL
FL
VVAFPASS
LFVYG
VAVALSALNLA
PN
EAIVG
FITVDGFN
LFGTIVGSL
VAAAASLALG
WFG
IAATTGFITGQL
AIGI
YGVAVAALG
VLL
ISILSDAGLG
AAT
GLNIGS
LESLGGL
YIIL
VPVLAPLPIGFAVF
TLLTIL
IEFT
TLLQ
FIAADTLQ
WFS
YVG
LPQFAGSAG
AIGV
TIL
ALGLSFLHL
LNILIAGPA
AIAFTLANT
FSLGL
WY
LGLFAF
LPQFSGSGG
ILPVNI
LTHAVGNVL
TVPVWAVP
ASAVQTVC
FWD
TIL
VQYYAYV
LFVPVTGLW
SAIGVVGLA
AVG
AIGPAAGGTILT
IGAITSGPI
VGAGVLSLPYA
NVSVN
FIAIPSFALL
TILQ
FAVGVNS
ISVGFNAAASV
IFATGVVLGSY
TGV
HSYVA
ALGGSALAVY
AYSSVSH
LVLLGSTTS
PLVTYAS
LVLLI
IEGA
VGGGYI
LVFLP
IVL
LY